# RDDs

### Creando RDDS

#### Primera forma

In [14]:
val stringList = Array("Jose","Aliannys","Idalmi")
val stringRDD = sc.parallelize(stringList)

stringList: Array[String] = Array(Jose, Aliannys, Idalmi)
stringRDD: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[10] at parallelize at <console>:28


#### Segunda  forma

The second way to create an RDD is to read a dataset from a storage system, which
can be a local computer file system, HDFS, Cassandra, Amazon S3, and so on.

In [15]:
val fileRDD = sc.textFile("test_RDD.txt")

fileRDD: org.apache.spark.rdd.RDD[String] = test_RDD.txt MapPartitionsRDD[12] at textFile at <console>:25


#### Tercera  forma

The third way to create an RDD is by invoking one of the transformation operations
on an existing RDD. Once you start becoming competent with Spark, you will do this all
the time without thinking twice about it.

### Transformaciones

#### map(func)

In [16]:
val namesUP = stringRDD.map(line => line.toUpperCase)
namesUP.collect().foreach(println)

JOSE
ALIANNYS
IDALMI


namesUP: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[13] at map at <console>:28


Otra vía que permite el mantenimiento del código

In [17]:
def mayuscula(line: String): String = {
    line.toUpperCase
}

stringRDD.map(l => mayuscula(l)).collect().foreach(println)

JOSE
ALIANNYS
IDALMI


mayuscula: (line: String)String


Otro ejemplo

In [18]:
val stringLengthRDD = stringRDD.map(l => l.length)
stringLengthRDD.collect().foreach(println)

4
8
6


stringLengthRDD: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[15] at map at <console>:28


#### flatMap(func)

In [19]:
val frases = Array("Jose lindo","Tuyo y mío", "solo tu y yo")
val frasesRDD = sc.parallelize(frases)

frases: Array[String] = Array(Jose lindo, Tuyo y mío, solo tu y yo)
frasesRDD: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[16] at parallelize at <console>:28


In [13]:
val wordRDD = frasesRDD.flatMap(l => l.split(" "))
wordRDD.collect().foreach(println)

Jose
lindo
Tuyo
y
mío
solo
tu
y
yo


wordRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[9] at flatMap at <console>:26


#### filter(func)

In [22]:
val joseRDD = stringRDD.filter(line => line.contains("Jose")).collect().foreach(println)

Jose


joseRDD: Unit = ()


#### mapPartitions(func)/mapPartitionsWithIndex(index, func)

In short, the mapPartitions and mapPartitionsWithIndex transformations are used
to optimize the performance of your data processing logic by reducing the number of
times the expensive setup step is called.

The next example first creates an RDD with two partitions and then creates a random
generator per partitions before iterating through each row. Finally, as it iterates through
the row, it adds a random number to each row in each partition in the RDD

In [26]:
import scala.util.Random

val sampleList = Array("One", "Two", "Three", "Four","Five")

val sampleRDD = spark.sparkContext.parallelize(sampleList, 2)

val result = sampleRDD.mapPartitions((itr:Iterator[String]) => {
    val rand = new Random(System.currentTimeMillis + Random.nextInt)
    itr.map(l => l + ":" + rand.nextInt)
    })
result.collect()

import scala.util.Random
sampleList: Array[String] = Array(One, Two, Three, Four, Five)
sampleRDD: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[19] at parallelize at <console>:28
result: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[20] at mapPartitions at <console>:30
res13: Array[String] = Array(One:-1620372493, Two:856580103, Three:1741214079, Four:1089910951, Five:1496680526)


Lo siguiente crea un objeto de tipo RANDOM y lo llama cada vez que se desee para generar números aleatorios

In [31]:
val rand = new Random(System.currentTimeMillis + Random.nextInt)
rand.nextInt

rand: scala.util.Random = scala.util.Random@32d907ad
res18: Int = 1939282200


Creating a Function to Encapsulate the Logic of Adding Random Numbers to Each Row

In [32]:
import scala.util.Random

def addRandomNumber(rows:Iterator[String]) = {
    val rand = new Random(System.currentTimeMillis + Random.nextInt)
    rows.map(l => l + " : " + rand.nextInt)
    }

import scala.util.Random
addRandomNumber: (rows: Iterator[String])Iterator[String]


Using the addRandomNumber Function in the mapPartitions
Transformation

In [33]:
val result = sampleRDD.mapPartitions((rows:Iterator[String]) => addRandomNumber(rows))

result: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[21] at mapPartitions at <console>:30


Using the mapPartitionsWithIndex Transformation

In [34]:
val numberRDD = spark.sparkContext.parallelize(List(1,2,3,4,5,6,7,8,9,10), 2)
numberRDD.mapPartitionsWithIndex((idx:Int, itr:Iterator[Int]) => {
    itr.map(n => (idx, n) )}).collect()

numberRDD: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[22] at parallelize at <console>:26
res19: Array[(Int, Int)] = Array((0,1), (0,2), (0,3), (0,4), (0,5), (1,6), (1,7), (1,8), (1,9), (1,10))


#### Operaciones de conjunto

#### union(otherRDD)

In [36]:
val rdd1 = sc.parallelize(Array(1,2,3,4,5))
val rdd2 = sc.parallelize(Array(1,6,7,8))
val rdd3 = rdd1.union(rdd2)
rdd3.collect()

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[24] at parallelize at <console>:27
rdd2: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[25] at parallelize at <console>:28
rdd3: org.apache.spark.rdd.RDD[Int] = UnionRDD[26] at union at <console>:29
res21: Array[Int] = Array(1, 2, 3, 4, 5, 1, 6, 7, 8)


#### intersection(otherRDD)

In [38]:
val rdd1 = sc.parallelize(Array(1,"jose"))
val rdd2 = sc.parallelize(Array(2,4,6,"jose"))
val rdd3 = rdd1.intersection(rdd2)
rdd3.collect()

rdd1: org.apache.spark.rdd.RDD[Any] = ParallelCollectionRDD[28] at parallelize at <console>:32
rdd2: org.apache.spark.rdd.RDD[Any] = ParallelCollectionRDD[29] at parallelize at <console>:33
rdd3: org.apache.spark.rdd.RDD[Any] = MapPartitionsRDD[35] at intersection at <console>:34
res22: Array[Any] = Array(jose)


#### substract(otherRDD)

Example: Removing Stop Words Using the subtract Transformation/no funciona substract!!!

In [40]:
val words = spark.sparkContext.parallelize(List("The amazing thing about spark is that it is very simple to learn"))
.flatMap(l => l.split(" "))
.map(w => w.toLowerCase)

val stopWords = spark.sparkContext.parallelize(List("the it is to that"))
.flatMap(l => l.split(" "))

val realWords = words.substract(stopWords)
realWords.collect()

<console>: 35: error: value substract is not a member of org.apache.spark.rdd.RDD[String]

#### distinct( )

In [43]:
val rdd1 = sc.parallelize(List(1,"uno","uno",2,3,3,2,"dos","tres"))
rdd1.distinct().collect()

rdd1: org.apache.spark.rdd.RDD[Any] = ParallelCollectionRDD[44] at parallelize at <console>:29
res25: Array[Any] = Array(dos, tres, 1, uno, 2, 3)


#### sample(withReplacement, fraction, seed)

* The withReplacement parameter determines whether an already sampled row will be placed back into RDD for the next sampling
* The given fraction value must be between 0 and 1, and it is not guaranteed that the returned RDD will have the exact fraction number of rows of the original RDD
* The optional seed parameter is used to seed the random generator, and it has a default value if one is not provided.

In [44]:
val rdd1 = sc.parallelize(List(1,2,3,4,5,6,7,8,9,10))
rdd1.sample(true, 0.4).collect()

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[48] at parallelize at <console>:29
res26: Array[Int] = Array(4, 10, 10)


### Acciones

#### collect( )

It collects all the rows from each of the partitions in an RDD and brings them over to the driver program. If your RDD contains 100 million rows, then it is not a good idea to invoke the collect action because the driver program most likely doesn’t have sufficient memory to hold all those rows. As a result, the driver will most likely run into an out-of-memory error and your Spark application or shell will die.

#### count( )

In [47]:
val rdd1 = sc.parallelize(Array(1,2,3,4,5,6,7,8,9,10), 2)
rdd1.count()

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[50] at parallelize at <console>:29
res29: Long = 10


#### first( )

In [48]:
val rdd1 = sc.parallelize(List(1,2,3,4,5))
rdd1.first()

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[51] at parallelize at <console>:29
res30: Int = 1


#### take(n)

In [49]:
val rdd1 = sc.parallelize(Array(12,3,21,2,34))
rdd1.take(3)

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[52] at parallelize at <console>:29
res31: Array[Int] = Array(12, 3, 21)


#### reduce(func)

*Ver ejemplo Listing 3.40-3.43 del libro Beginning Apache Spark 2_ With Resilient Distributed Datasets, Spark SQL, Structured Streaming and Spark Machine Learning library (2018, Apress)*

#### takeSample(withReplacement, n, [seed])

The behavior of this action is similar to the behavior of the sample transformation. The main difference is this action returns an array of sampled rows to the driver program. The same caution for the collect action is applicable here in terms of the large number of returned rows.

#### takeOrdered(n, [ordering])

This action returns n rows in a certain order. The default ordering for this action is the natural ordering. If the rows are integers, then the default ordering is ascending. If you need to return n rows with the values in descending order, then you specify the reverse ordering.

In [50]:
val rdd1 = sc.parallelize(Array(1,2,3,4,5,6,7,9,10), 2)
rdd1.takeOrdered(4)

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[53] at parallelize at <console>:29
res32: Array[Int] = Array(1, 2, 3, 4)


In [51]:
rdd1.takeOrdered(4)(Ordering[Int].reverse)

res33: Array[Int] = Array(10, 9, 7, 6)


#### top(n, [ordering])

A good use case for using this action is for figure out the top k (largest) rows in an RDD as defined by the implicit ordering.

In [52]:
val rdd1 = sc.parallelize(List(1,2,33,-1,45,67,-111), 2)
rdd1.top(3)

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[56] at parallelize at <console>:29
res34: Array[Int] = Array(67, 45, 33)


#### saveAsTextFile(path)

In [56]:
val rdd1 = sc.parallelize(Array(12,54,32,45,6,2,3,4,56,7,45,3,2,1), 3)
rdd1.saveAsTextFile("./save_as_text_RDD.txt")

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[64] at parallelize at <console>:29


### Creating Key/Value Pair RDD

In [57]:
val rdd = sc.parallelize(List("Spark","is","an", "amazing", "piece","of","technology"))
val pairRDD = rdd.map(w => (w.length,w))
pairRDD.collect().foreach(println)

(5,Spark)
(2,is)
(2,an)
(7,amazing)
(5,piece)
(2,of)
(10,technology)


rdd: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[66] at parallelize at <console>:27
pairRDD: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[67] at map at <console>:28


#### Key/Value Pair RDD Transformations

A key/value pair RDD has additional transformations that are designed to operate on keys. See **Table 3-4** *Common Transformations for Pair RDD*

#### Key/Value Pair RDD Actions

See **Table 3-5** *Actions for Pair RDD*